<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/timesformer/TimeSformer-huggingface-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정

In [ ]:
!pip install torch torchvision
!pip install timm
!pip install pytorchvideo
!pip install torchvision==0.15.1

# 라이브러리 임포트

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from pytorchvideo.data import Kinetics
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    RandomShortSideScale,
    Normalize,
)
from torchvision.transforms import (
    RandomCrop,
    RandomHorizontalFlip
)
# from pytorchvideo.models.hub import timesformer
import timm

# 데이터셋 로드 및 전처리

In [ ]:
def make_kinetics_dataloader(data_path, video_path_prefix, batch_size, num_workers=4):
    transform = transforms.Compose([
        ApplyTransformToKey(
            key="video",
            transform=transforms.Compose([
                RandomShortSideScale(min_size=256, max_size=320),
                RandomCrop(224),
                RandomHorizontalFlip(),
                Normalize(
                    mean=[0.45, 0.45, 0.45],
                    std=[0.225, 0.225, 0.225]
                )
            ])
        )
    ])

    dataset = Kinetics(
        data_path=data_path,
        clip_sampler="uniform",
        video_sampler=torch.utils.data.RandomSampler,
        transform=transform,
        video_path_prefix=video_path_prefix,
        decode_audio=False
    )

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True
    )
    return dataloader

train_dataloader = make_kinetics_dataloader('path/to/kinetics/train', 'path/to/kinetics/train', batch_size=8)
val_dataloader = make_kinetics_dataloader('path/to/kinetics/val', 'path/to/kinetics/val', batch_size=8)


FileNotFoundError: path/to/kinetics/train not found.

In [ ]:
Kinetics?

In [1]:
from transformers import AutoImageProcessor, TimesformerForVideoClassification
import numpy as np
import torch

video = np.random.randn(8, 3, 224, 224) # 8개 프레임은 3채널(RGB)과 224x224
video = (video - video.min()) / (video.max() - video.min())  # Normalize to [0, 1]
video = list(video)  # 리스트로 변환

processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400") # 입력 비디오를 모델이 처리할 수 있는 텐서로 변환하는 processor
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

inputs = processor(video, return_tensors="pt")

with torch.no_grad():
  outputs = model(**inputs)
  logits = outputs.logits

predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/486M [00:00<?, ?B/s]

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)


Predicted class: singing


## Kinetics-400 데이터셋의 예제 비디오 다운로드  


In [2]:
!rm -rf archery*

# !wget https://github.com/pytorch/vision/blob/master/test/assets/videos/R6llTwEh07w.mp4 -O archery-raw.mp4
!wget https://github.com/pytorch/vision/blob/master/test/assets/videos/SOX5yA1l24A.mp4 -O archery-raw.mp4
!ls

--2024-06-17 08:09:30--  https://github.com/pytorch/vision/blob/master/test/assets/videos/SOX5yA1l24A.mp4
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/pytorch/vision/blob/main/test/assets/videos/SOX5yA1l24A.mp4 [following]
--2024-06-17 08:09:31--  https://github.com/pytorch/vision/blob/main/test/assets/videos/SOX5yA1l24A.mp4
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘archery-raw.mp4’

archery-raw.mp4         [  <=>               ] 271.52K  1.09MB/s    in 0.2s    

2024-06-17 08:09:32 (1.09 MB/s) - ‘archery-raw.mp4’ saved [278041]

archery-raw.mp4  sample_data


In [3]:
!apt-get remove -y ffmpeg

# ffmpeg 3.4.2 다운로드
!wget https://www.johnvansickle.com/ffmpeg/old-releases/ffmpeg-3.4.2-64bit-static.tar.xz

# 압축 해제
!tar -xJf ffmpeg-3.4.2-64bit-static.tar.xz

# ffmpeg 3.4.2 설치
!cp ffmpeg-3.4.2-64bit-static/ffmpeg /usr/bin/ffmpeg
!cp ffmpeg-3.4.2-64bit-static/ffprobe /usr/bin/ffprobe

# 설치 확인
!ffmpeg -version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  ffmpeg
0 upgraded, 0 newly installed, 1 to remove and 45 not upgraded.
After this operation, 2,288 kB disk space will be freed.
(Reading database ... 121913 files and directories currently installed.)
Removing ffmpeg (7:4.4.2-0ubuntu0.22.04.1) ...
Processing triggers for man-db (2.10.2-1) ...
--2024-06-17 08:09:41--  https://www.johnvansickle.com/ffmpeg/old-releases/ffmpeg-3.4.2-64bit-static.tar.xz
Resolving www.johnvansickle.com (www.johnvansickle.com)... 107.180.57.212
Connecting to www.johnvansickle.com (www.johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17278680 (16M) [application/x-xz]
Saving to: ‘ffmpeg-3.4.2-64bit-static.tar.xz’

ffmpeg-3.4.2-64bit- 100%[===================>]  16.48M  10.1MB/s    in 1.6s    

2024-06-17 08:09:44 (10.1 MB/s) - ‘ffmpeg-3.4.2-64bit-static.tar.xz’ saved

In [4]:
!ls
!ffmpeg -i archery-raw.mp4
!ffmpeg -i archery-raw.mp4 -c copy -movflags faststart archery-fixed.mp4
# !ffmpeg -i archery-raw.mp4 -vcodec libx264 -acodec aac archery.mp4

archery-raw.mp4  ffmpeg-3.4.2-64bit-static  ffmpeg-3.4.2-64bit-static.tar.xz  sample_data
ffmpeg version 3.4.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 6.3.0 (Debian 6.3.0-18) 20170516
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc-6 --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gray --enable-libfribidi --enable-libass --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libzimg
  libavutil      55. 78.100 / 55. 78.100
  libavcodec     57.107.100 / 57.107.100
  libavformat    57. 83.100 / 57. 83.100

In [5]:
!ls

archery-raw.mp4  ffmpeg-3.4.2-64bit-static  ffmpeg-3.4.2-64bit-static.tar.xz  sample_data


In [6]:
import cv2
from transformers import AutoImageProcessor, TimesformerForVideoClassification
import torch
import matplotlib.pyplot as plt

# 비디오 파일을 로드하고 프레임을 추출
def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)
        frame_count += 1
        if frame_count <= 2:  # 처음 2개의 프레임 출력
            print(f"Frame {frame_count} shape: {frame.shape}")
    cap.release()
    print(f"Total frames extracted: {len(frames)}")
    return frames

In [12]:
import os

# 비디오 파일 존재 확인
# video_path = './archery-raw.mp4'

# if os.path.exists(video_path):
#     print(f"{video_path} exists.")
# else:
#     print(f"{video_path} does not exist.")

# frames = load_video(video_path)

# frames = torch.randn(3, 8, 224, 224) # (batch x channels x frames x height x width)
video = np.random.randn(8, 3, 224, 224) # 8개 프레임은 3채널(RGB)과 224x224
video = (video - video.min()) / (video.max() - video.min())  # Normalize to [0, 1]
video = list(video)  # 리스트로 변환

# 첫 8개의 프레임 사용
# video = frames[:8]

print(f"Video shape: {len(video)} frames")
print(f"Each frame shape: {video[0].shape}")



Video shape: 8 frames
Each frame shape: (3, 224, 224)


In [13]:
# 이미지 프로세서와 모델 로드
processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

# 입력 데이터 전처리
inputs = processor([video], return_tensors="pt")

# 모델 추론
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# 예측된 클래스 추출 및 출력
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: singing
